# Hand Labelled Homography
Generate a pipeline for quantitative analysis of homography estimation. Do
 - Randomly sample sequences from list of videos
 - Safe sequences into folder
 - Annotate some sequences
 - Create evaluation pipeline, precision, drift


In [3]:
import sys
import pandas as pd
from dotmap import DotMap

sys.path.append('../')

from utils.io import load_yaml

server = 'local'
servers = load_yaml('../config/servers.yml')
server = DotMap(servers[server])

meta_df = pd.read_pickle('../config/cholec80_transforms.pkl')
meta_df

,database,train,file,pre_transforms,aug_transforms,auxiliary
0,cholec80,True,"{'name': 'video01.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
1,cholec80,True,"{'name': 'video02.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
2,cholec80,True,"{'name': 'video03.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
3,cholec80,True,"{'name': 'video04.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
4,cholec80,True,"{'name': 'video05.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
...,...,...,...,...,...,...
70,cholec80,True,"{'name': 'video17.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
71,cholec80,True,"{'name': 'video18.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
72,cholec80,True,"{'name': 'video19.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
73,cholec80,True,"{'name': 'video20.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}


## Randomly Sample Image Sequences

In [10]:
import cv2
import numpy as np
from kornia import tensor_to_image

from utils.transforms import anyDictListToCompose
from utils.sampling import RandomSequences
from utils.io import generate_path

debug = False

max_seq = 10
paths = meta_df.apply(lambda x: os.path.join(server.database.location, x.database, x.file['path'], x.file['name']), axis=1).tolist()
seq_len = 20
strides = [5]

# append to tensor transform, as meta_df is supposed to operate on tensors
to_tensor = {'module': 'torchvision.transforms', 'type': 'ToTensor', 'kwargs': {}}
transforms = meta_df.apply(lambda x: anyDictListToCompose([to_tensor] + x.pre_transforms), axis=1).tolist()

random_sequences = RandomSequences(
    max_seq=max_seq,
    paths=paths,
    seq_len=seq_len,
    strides=strides,
    transforms=transforms,
    verbose=True
)

out_prefix = 'out/homography_labelling'

for seq, vid_idx, frame_idx in random_sequences:
    print('vid_idx: {}, frame_idx: {}'.format(vid_idx, frame_idx))
    for idx, frame in enumerate(seq):
        frame = (tensor_to_image(frame)*255).astype(np.uint8)
        if debug:
            cv2.imshow('random_frame', frame)  # show images
            cv2.waitKey()
        else:
            vid_path = os.path.join(out_prefix, 'vid_{}_frame_{}'.format(vid_idx, frame_idx))
            generate_path(vid_path)
            cv2.imwrite(os.path.join(vid_path, 'frame_{}.png'.format(frame_idx + idx*strides[0])), frame)
cv2.destroyAllWindows()

vid_idx: 60, frame_idx: 10966
vid_idx: 58, frame_idx: 10791
vid_idx: 6, frame_idx: 53189
vid_idx: 36, frame_idx: 14944
vid_idx: 14, frame_idx: 15351
vid_idx: 22, frame_idx: 27941
vid_idx: 56, frame_idx: 68257
vid_idx: 30, frame_idx: 8978
vid_idx: 8, frame_idx: 10455
vid_idx: 0, frame_idx: 5980


### Dataset on Sampled Frames

In [2]:
import os

from utils.io import recursive_scan2df

prefix = 'out/homography_labelling'

# create a simplified dataframe
postfix = '.png'
df = recursive_scan2df(prefix, postfix)
df['vid'] = df.folder.apply(lambda x: int(x.split('_')[-3]))
df['frame'] = df.file.apply(lambda x: int(x.split('_')[-1].replace(postfix, '')))
df = df.sort_values(['vid', 'frame']).reset_index(drop=True)

out_path = 'light_log'
df.to_pickle(os.path.join(prefix, out_path + '.pkl'))

df

,folder,file,vid,frame
0,vid_0_frame_5980,frame_5980.png,0,5980
1,vid_0_frame_5980,frame_5985.png,0,5985
2,vid_0_frame_5980,frame_5990.png,0,5990
3,vid_0_frame_5980,frame_5995.png,0,5995
4,vid_0_frame_5980,frame_6000.png,0,6000
...,...,...,...,...
175,vid_60_frame_10966,frame_11041.png,60,11041
176,vid_60_frame_10966,frame_11046.png,60,11046
177,vid_60_frame_10966,frame_11051.png,60,11051
178,vid_60_frame_10966,frame_11056.png,60,11056


## Estimate Homographies on Sequences

## Hand Labelled

In [1]:
import pandas as pd
import numpy as np
import sys
import os
import cv2

sys.path.append('../')

from utils.io import recursive_scan2df
from utils.viz import yt_alpha_blend

prefix = 'out/homography_labelling'
vid_df = pd.read_pickle(os.path.join(prefix, 'light_log.pkl'))
pts_df = recursive_scan2df(prefix, '.txt')

for _, row in pts_df.iterrows():
    df = pd.read_csv(os.path.join(prefix, row.folder, row.file), delimiter='\t', header=None, names=['x', 'y']) # stored as x, y opencv convention
    n_img = len(vid_df[vid_df.folder == row.folder])
    n_pts = int(len(df) / n_img)

    print('N Points: {}'.format(n_pts))
    
    # get homography
    for img_idx in range(n_img - 1):
        src_pts = df.iloc[img_idx*n_pts:(img_idx+1)*n_pts]
        dst_pts = df.iloc[(img_idx+1)*n_pts:(img_idx+2)*n_pts]
        
        src_pts = np.array([[pt.x, pt.y] for _, pt in src_pts.iterrows()])
        dst_pts = np.array([[pt.x, pt.y] for _, pt in dst_pts.iterrows()])

        H, _ = cv2.findHomography(src_pts, dst_pts)
        
        # show images
        img = cv2.imread(os.path.join(prefix, vid_df[vid_df.folder == row.folder].iloc[img_idx].folder, vid_df[vid_df.folder == row.folder].iloc[img_idx].file))
        wrp = cv2.imread(os.path.join(prefix, vid_df[vid_df.folder == row.folder].iloc[img_idx + 1].folder, vid_df[vid_df.folder == row.folder].iloc[img_idx + 1].file))
        cv2.imshow('img', img)
        cv2.imshow('wrp', wrp)

        # create blend
        wrp_pred = cv2.warpPerspective(img, H, (img.shape[1], img.shape[0]))
        cv2.imshow('wrp_pred', wrp_pred)
        blend = yt_alpha_blend(wrp, wrp_pred)
        cv2.imshow('blend', blend/255)  # note that more points are needed, or blend will only work locally around pts
        cv2.waitKey()

cv2.destroyAllWindows()

7


### Forward-Backward Consistency

In [4]:
import cv2
import torch
import numpy as np
import pandas as pd
import torch.functional as F
from kornia import tensor_to_image, warp_perspective
from torch.utils.data import DataLoader
import sys

sys.path.append('../')

from utils.viz import yt_alpha_blend
from utils.transforms import anyDictListToCompose
from utils.processing import forward_backward_sequence, image_edges, four_point_homography_to_matrix
from datasets import ImageSequenceDataset
from lightning_modules import DeepImageHomographyEstimationModuleBackbone

def visualize(fw_img, fw_wrp, bw_img, bw_wrp, fw_duv, bw_duv):
    fw_uv, bw_uv = image_edges(fw_img), image_edges(bw_img)
    fw_H, bw_H = four_point_homography_to_matrix(fw_uv, fw_duv), four_point_homography_to_matrix(bw_uv, bw_duv)

    fw_pred_wrp = warp_perspective(fw_img, torch.inverse(fw_H), fw_img.shape[-2:])
    bw_pred_wrp = warp_perspective(bw_img, torch.inverse(bw_H), bw_img.shape[-2:])
    
    fw_blend, bw_blend = yt_alpha_blend(fw_wrp, fw_pred_wrp), yt_alpha_blend(bw_wrp, bw_pred_wrp)
    fw_blend, bw_blend = tensor_to_image(fw_blend), tensor_to_image(bw_blend)

    cv2.imshow('fw_blend', fw_blend)
    cv2.imshow('bw_blend', bw_blend)
    cv2.waitKey()

prefix = 'out/homography_labelling'
# prefix = '/media/martin/Samsung_T5/data/endoscopic_data/camera_motion_separated_png/without_camera_motion'
df = pd.read_pickle(os.path.join(prefix, 'light_log.pkl'))
# df = pd.read_pickle(os.path.join(prefix, 'light_log_without_camera_motion.pkl'))
seq_len = 2

transforms = [
    {'module': 'torchvision.transforms', 'type': 'ConvertImageDtype', 'kwargs': {'dtype': torch.float}}
]
transforms = [anyDictListToCompose(transforms) for _ in range(len(meta_df))]

ds = ImageSequenceDataset(
    df=df,
    prefix=prefix,
    seq_len=seq_len,
    transforms=transforms
)

# load network
prefix = '/home/martin/Tresors/homography_imitation_learning_logs/deep_image_homography_estimation_backbone/version_2'
configs = load_yaml(os.path.join(prefix, 'configs.yml'))
model = DeepImageHomographyEstimationModuleBackbone.load_from_checkpoint(os.path.join(prefix, 'checkpoints/epoch=49.ckpt'), shape=configs['model']['shape'])

device = 'cpu'
if torch.cuda.is_available():
    print('Running with CUDA backend.')
    device = 'cuda'

model.to(device)
model = model.eval()
model.freeze()


batch_size = 1
num_workers = 0

dl = DataLoader(ds, batch_size=batch_size, num_workers=num_workers)

fw_mean_distance = []
bw_mean_distance = []
con_mean_distance = []

I = 100
for idx, batch in enumerate(dl):
    fw, bw = forward_backward_sequence(batch)
    print(fw.shape, bw.shape)
   

    # create pairs
    fw_img, fw_wrp = fw[:,:-1:].view((-1,) + fw.shape[-3:]).to(device), fw[:,1::].view((-1,) + fw.shape[-3:]).to(device) 
    bw_img, bw_wrp = bw[:,:-1:].view((-1,) + bw.shape[-3:]).to(device), bw[:,1::].view((-1,) + bw.shape[-3:]).to(device)

    # fw_img, fw_wrp, bw_img, bw_wrp = tensor_to_image(fw_img[0]), tensor_to_image(fw_wrp[0]), tensor_to_image(bw_img[0]), tensor_to_image(bw_wrp[0])
    # cv2.imshow('fw_img', fw_img)
    # cv2.imshow('fw_wrp', fw_wrp)
    # cv2.imshow('bw_img', bw_img)
    # cv2.imshow('bw_wrp', bw_wrp)
    # cv2.waitKey()

    fw_duv = model(fw_img, fw_wrp)
    bw_duv = model(bw_img, bw_wrp)

    # compute error, sum isnt that great of a measurement as 2*N vs N elements
    fw_mean_distance.append(torch.linalg.norm(fw_duv.sum(axis=0), axis=1).mean().item())
    bw_mean_distance.append(torch.linalg.norm(bw_duv.sum(axis=0), axis=1).mean().item())
    duv = torch.cat((fw_duv, bw_duv))
    con_mean_distance.append(torch.linalg.norm(duv.sum(axis=0), axis=1).mean().item())

    if idx == I - 1:
        break

    visualize(fw_img, fw_wrp, bw_img, bw_wrp, fw_duv, bw_duv)

fw_mean_distance = np.array(fw_mean_distance)
bw_mean_distance = np.array(bw_mean_distance)
con_mean_distance = np.array(con_mean_distance)

print(fw_mean_distance.mean(), fw_mean_distance.var())
print(bw_mean_distance.mean(), bw_mean_distance.var())
print(con_mean_distance.mean(), con_mean_distance.var())
    
cv2.destroyAllWindows()
    

    

Running with CUDA backend.
torch.Size([1, 2, 3, 480, 640]) torch.Size([1, 2, 3, 480, 640])
torch.Size([1, 2, 3, 480, 640]) torch.Size([1, 2, 3, 480, 640])
torch.Size([1, 2, 3, 480, 640]) torch.Size([1, 2, 3, 480, 640])
torch.Size([1, 2, 3, 480, 640]) torch.Size([1, 2, 3, 480, 640])
torch.Size([1, 2, 3, 480, 640]) torch.Size([1, 2, 3, 480, 640])
torch.Size([1, 2, 3, 480, 640]) torch.Size([1, 2, 3, 480, 640])
torch.Size([1, 2, 3, 480, 640]) torch.Size([1, 2, 3, 480, 640])
torch.Size([1, 2, 3, 480, 640]) torch.Size([1, 2, 3, 480, 640])
torch.Size([1, 2, 3, 480, 640]) torch.Size([1, 2, 3, 480, 640])
torch.Size([1, 2, 3, 480, 640]) torch.Size([1, 2, 3, 480, 640])
torch.Size([1, 2, 3, 480, 640]) torch.Size([1, 2, 3, 480, 640])
torch.Size([1, 2, 3, 480, 640]) torch.Size([1, 2, 3, 480, 640])
torch.Size([1, 2, 3, 480, 640]) torch.Size([1, 2, 3, 480, 640])
torch.Size([1, 2, 3, 480, 640]) torch.Size([1, 2, 3, 480, 640])
torch.Size([1, 2, 3, 480, 640]) torch.Size([1, 2, 3, 480, 640])
torch.Size([1